<a href="https://colab.research.google.com/github/FraCorti/AI_Project/blob/master/spamFilter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Added **Kaggle** installation and downloading *spam.csv* dataset from the following link: [spam-collection-dataset](https://www.kaggle.com/uciml/sms-spam-collection-dataset)

In [3]:
!pip install kaggle
!mkdir /root/.kaggle
!echo '{"username":"fcorti","key":"f6cb1b6a4e144d900623ca8d446b0fa3"}' > /root/.kaggle/kaggle.json
!kaggle datasets download -d uciml/sms-spam-collection-dataset
!pip install pandas
!pip install -q tensorflow_data_validation

import tensorflow_data_validation as tfdv
import pandas as pd
import tensorflow as tf

from zipfile import ZipFile

  0% 0.00/208k [00:00<?, ?B/s]
100% 208k/208k [00:00<00:00, 58.9MB/s]
     |████████████████████████████████| 2.7MB 4.8MB/s 
     |████████████████████████████████| 2.8MB 43.7MB/s 
     |████████████████████████████████| 174kB 49.0MB/s 
     |████████████████████████████████| 61kB 20.9MB/s 
     |████████████████████████████████| 51kB 20.4MB/s 
     |████████████████████████████████| 48.5MB 1.4MB/s 
     |████████████████████████████████| 153kB 42.9MB/s 
     |████████████████████████████████| 81kB 22.8MB/s 
     |████████████████████████████████| 133kB 43.5MB/s 
     |████████████████████████████████| 1.2MB 43.7MB/s 
     |████████████████████████████████| 163kB 49.6MB/s 
     |████████████████████████████████| 174kB 48.9MB/s 
     |████████████████████████████████| 92kB 26.8MB/s 
     |████████████████████████████████| 102kB 28.9MB/s 
     |████████████████████████████████| 92kB 27.6MB/s 
     |████████████████████████████████| 112kB 48.0MB/s 
ERROR: multiprocess 0.70.8 has requireme

/usr/local/lib/python3.6/dist-packages/apache_beam/__init__.py:84: UserWarning: Running the Apache Beam SDK on Python 3 is not yet fully supported. You may encounter buggy behavior or missing features.
  'Running the Apache Beam SDK on Python 3 is not yet fully supported. '


Imported **pandas** and convert the *sms-spam-collection-dataset.zip* file into *.csv* format


In [4]:
zip_file = ZipFile('/content/sms-spam-collection-dataset.zip')
data = pd.read_csv(zip_file.open('spam.csv'), encoding = "latin-1")
data = data.drop(["Unnamed: 2", "Unnamed: 3", "Unnamed: 4"], axis=1)
data = data.rename(columns={"v1":'label', "v2":'text'})
tags = data["label"]
texts = data["text"]  
print("Dataset shape: ", data.shape)

Dataset shape:  (5572, 2)


Save the dataset **locally** to allow viewing by clicking on it: 

In [0]:
data.to_csv(r'/content/spam.csv', index = None, header = True)

Dataset **statistics**: 

In [6]:
train_stats = tfdv.generate_statistics_from_csv(data_location = '/content/spam.csv')
tfdv.visualize_statistics(train_stats)

W0728 14:14:34.095588 139896949577600 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tensorflow_data_validation/utils/stats_gen_lib.py:144: The name tf.gfile.Exists is deprecated. Please use tf.io.gfile.exists instead.

W0728 14:14:34.110872 139896949577600 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tensorflow_data_validation/utils/stats_gen_lib.py:292: The name tf.gfile.Glob is deprecated. Please use tf.io.gfile.glob instead.

W0728 14:14:34.116199 139896949577600 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tensorflow_data_validation/utils/stats_gen_lib.py:298: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.

W0728 14:14:34.209892 139896949577600 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tensorflow_transform/analyzers.py:948: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

W0728 14:14:34.439790 139896949577600 depreca

**Divided data** into training set and test set: 

1.   List item
2.   List item



In [7]:
!pip install sklearn

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Conv1D, GlobalMaxPooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras import metrics

num_max = 1000  
le = LabelEncoder()  
tags = le.fit_transform(tags) 
tokenizer = Tokenizer(num_words = num_max)  # allows to vectorize a text corpus, by turning each text into either a sequence of integers or into a vector.
tokenizer.fit_on_texts(texts) # updates internal vocabulary based on a list of texts. Creates the vocabulary index based on word frequency.
mat_texts = tokenizer.texts_to_matrix(texts, mode = 'count')  # count: the count of each word in the document.

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(texts , tags, test_size = 0.2)
mat_texts_tr = tokenizer.texts_to_matrix(x_train, mode = 'count')
mat_texts_tst = tokenizer.texts_to_matrix(x_test, mode = 'count')

print (x_train.shape, y_train.shape) # print the train dataset shape 
print (x_test.shape, y_test.shape)   # print the test dataset shape

(4457,) (4457,)
(1115,) (1115,)


Create the  **Sequential Keras** model, more info at ["The Sequential model API"](https://keras.io/models/sequential/). \\
More info about the *Dropout technique* can be found in the following paper: ["Dropout: A Simple Way to Prevent Neural Networks from
Overfitting"](http://www.jmlr.org/papers/volume15/srivastava14a/srivastava14a.pdf)

In [14]:
    sequentialModel = Sequential([
        Dense(512, input_dim = num_max),
        Activation('relu'),
        Dropout(0.2),
        Dense(256),
        Activation('relu'),
        Dropout(0.2),
        Dense(1),
        Activation('sigmoid')
    ])
    model.summary()      

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               512512    
_________________________________________________________________
activation (Activation)      (None, 512)               0         
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               131328    
_________________________________________________________________
activation_1 (Activation)    (None, 256)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 2

**TensorBoard setup**. \\
See tutorial: ["Get started with TensorBoard"](https://www.tensorflow.org/tensorboard/r2/get_started) if you want more info about the setup used. \\
More info about TensorBoard can be found in the official documentation: ["TensorBoard: Visualizing Learning"](https://www.tensorflow.org/guide/summaries_and_tensorboard).

In [12]:
# Load the TensorBoard notebook extension

%load_ext tensorboard.notebook
import datetime

# Clear any logs from previous runs

!rm -rf ./logs/ 
log_dir="logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

RuntimeError: ignored

**Configure** the learning process, more info at ["Compilation"](https://keras.io/getting-started/sequential-model-guide/#compilation)

In [15]:
sequentialModel.compile(loss = 'binary_crossentropy',
              optimizer = 'adam', 
              metrics = ['acc', metrics.binary_accuracy]) 

W0728 14:17:27.327503 139896949577600 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


**Display** the model and saves it in a *.png* format file 

In [21]:
from keras.utils import plot_model
from IPython.display import Image

plot_model(model, to_file = 'model.png')
Image('model.png')

TypeError: ignored

**Training** the model with the 80% of the spam dataset, more info at  ["Training"](https://keras.io/getting-started/sequential-model-guide/#training) 

In [18]:
    # verbose:  shows the progress bar during training see keras documentation for more info 
    # batch size: defines the number of samples that will be propagated through the network
    # epochs: iterations on a dataset
    
sequentialModel.fit(mat_texts_tr,
                    y_train, 
                    batch_size = 32, 
                    epochs = 10, 
                    verbose = 1, 
                    validation_split= 0.2)

Train on 3565 samples, validate on 892 samples
Epoch 1/10
3565/3565 [==============================] - 1s 379us/sample - loss: 0.1777 - acc: 0.9428 - binary_accuracy: 0.9428 - val_loss: 0.0439 - val_acc: 0.9865 - val_binary_accuracy: 0.9865
Epoch 2/10
3565/3565 [==============================] - 1s 237us/sample - loss: 0.0327 - acc: 0.9910 - binary_accuracy: 0.9910 - val_loss: 0.0476 - val_acc: 0.9877 - val_binary_accuracy: 0.9877
Epoch 3/10
3565/3565 [==============================] - 1s 240us/sample - loss: 0.0097 - acc: 0.9975 - binary_accuracy: 0.9975 - val_loss: 0.0473 - val_acc: 0.9888 - val_binary_accuracy: 0.9888
Epoch 4/10
3565/3565 [==============================] - 1s 225us/sample - loss: 0.0026 - acc: 0.9994 - binary_accuracy: 0.9994 - val_loss: 0.0561 - val_acc: 0.9888 - val_binary_accuracy: 0.9888
Epoch 5/10
3565/3565 [==============================] - 1s 225us/sample - loss: 0.0012 - acc: 1.0000 - binary_accuracy: 1.0000 - val_loss: 0.0599 - val_acc: 0.9888 - val_binary_

**Test** the neural network with the test data:

In [19]:
  sequentialModel.evaluate(mat_texts_tst, y_test)

1115/1115 [==============================] - 0s 83us/sample - loss: 0.1619 - acc: 0.9883 - binary_accuracy: 0.9883


[0.1619271223933693, 0.9883408, 0.9883408]

**TensorBoard** training data, plot **training** and **validation** accuracy values:

In [20]:
%tensorboard --logdir logs/fit

UsageError: Line magic function `%tensorboard` not found.
